# Import libs

In [ ]:
# Warning control
import warnings
import os
import yaml

from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool

from pydantic import BaseModel, Field
# from IPython.display import display, Markdown


load_dotenv()  # take environment variables from .env
warnings.filterwarnings("ignore")

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Loading Tasks and Agents YAML files

In [4]:
# Define file paths for YAML configurations
files = {
    "agents": "src/recsys/config/agents.yaml",
    "tasks": "src/recsys/config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

## Using FileReadTool

In [ ]:
csv_tool = FileReadTool(file_path="data/support_tickets_data.csv")
